In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import glob
import time
import pytz
import datetime as dt
import dateutil
from itertools import chain
from cycler import cycler
from io import StringIO


import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pvlib as pvlib
import seaborn as sns


ONE_DAY = dt.timedelta(days=1)

In [2]:
# Style of figures
plt.style.use('seaborn-poster')
plt.rcParams['figure.figsize'] = 15, 6.5
# get colormap
#cmap=plt.cm.hsv
cmap =  plt.cm.gist_ncar
#cmap=plt.cm.brg
# build cycler with 10 equally spaced colors from that colormap
c = cycler('color', cmap(np.linspace(0,1,17)) )
# supply cycler to the rcParam
plt.rcParams["axes.prop_cycle"] = c

In [3]:
def find_files(date_1, date_2, mask, bdd_dir):

    tmp_date = date_1
    list_files = []
    while tmp_date <= date_2:
        
        in_dir = bdd_dir.format(tmp_date)
        tmp_file = mask.format(tmp_date)
        tmp_file = os.path.join(in_dir, tmp_file)
        #print(tmp_file)
        tmp_list = sorted(glob.glob(tmp_file))
        #print(tmp_list)
    
        if len(tmp_list) != 0:
            list_files.append(tmp_list)
        
        tmp_date += ONE_DAY
    
    return sorted([file_ for file_ in chain.from_iterable(list_files)])


# Battery Charging data sets

In [4]:
folder=''
#filename='Real_Time_Saving_{:%Y-%m-%d}*.txt'
filename='/Users/jithinjomy/Downloads/PHY661k/data1.csv'
#filename='/Users/jbadosa/Downloads/Drahi*.csv'


files=glob.glob(filename)
print(files)


['/Users/jithinjomy/Downloads/PHY661k/data1.csv']


In [5]:
data = []
data = pd.read_csv(
    filename, # nom du fichier
    sep=',',  # le caractère utilisé comme séparateur de colonne
    header=0,  # la ligne contenant le titre des colonnes
    infer_datetime_format=True,  # on le laisse tenter de décoder les dates seules
    parse_dates=['Time Stamp'],  # on indique les colonnes contenant des dates
    index_col=['Time Stamp'],  # la colonne jouant le role "d'index", la colonne principale
)

In [ ]:
data.tail(4)

In [6]:
data.columns

Index(['Step', 'Status', 'Prog Time', 'Step Time', 'Cycle', 'Cycle Level',
       'Procedure', 'Voltage', 'Current', 'Temperature', 'Capacity', 'WhAccu',
       'Cnt'],
      dtype='object')

In [ ]:
#path = "/Users/jithinjomy/Downloads/PHY661k/data1" 

#all_files = glob.glob(os.path.join(path, "551_Charge*.csv"))
#df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
#df_merged   = pd.concat(df_from_each_file, ignore_index=True)
#df_merged.to_csv( "merged.csv")



In [ ]:
#df_merged['Capacity'] = df_merged['Capacity'].map(lambda x: x.strip('[').strip('Ah').strip(']'))
#df_merged.replace(to_replace ='',value ="0")

In [ ]:
#data.drop([], axis = 0, inplace = True)

In [7]:
data['SOC'] = data.Capacity/3
data.SOC

Time Stamp
2018-10-29 11:15:57    0.000000
2018-10-29 11:16:57    0.000000
2018-10-29 11:17:57    0.000000
2018-10-29 11:18:57    0.000000
2018-10-29 11:19:57    0.000000
                         ...   
2018-10-30 01:52:30    0.871477
2018-10-30 01:53:30    0.871477
2018-10-30 01:54:30    0.871477
2018-10-30 01:54:30    0.871477
2018-10-30 01:54:30    0.000000
Name: SOC, Length: 2906, dtype: float64

In [8]:
def_mergedX = data[['Voltage','Current']]
def_mergedX


,Voltage,Current
Time Stamp,,
2018-10-29 11:15:57,3.22633,0.0
2018-10-29 11:16:57,3.22734,0.0
2018-10-29 11:17:57,3.22835,0.0
2018-10-29 11:18:57,3.22919,0.0
2018-10-29 11:19:57,3.23003,0.0
...,...,...
2018-10-30 01:52:30,4.18596,0.0
2018-10-30 01:53:30,4.18579,0.0
2018-10-30 01:54:30,4.18579,0.0


In [9]:
def_mergedY = data[['SOC']]
def_mergedY

,SOC
Time Stamp,
2018-10-29 11:15:57,0.000000
2018-10-29 11:16:57,0.000000
2018-10-29 11:17:57,0.000000
2018-10-29 11:18:57,0.000000
2018-10-29 11:19:57,0.000000
...,...
2018-10-30 01:52:30,0.871477
2018-10-30 01:53:30,0.871477
2018-10-30 01:54:30,0.871477


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(def_mergedX, def_mergedY, test_size=0.2, random_state=101)


In [11]:
X_train.to_numpy(dtype = None)

array([[3.242  , 0.     ],
       [3.23661, 0.     ],
       [4.19995, 2.4545 ],
       ...,
       [3.24605, 0.     ],
       [3.24436, 0.     ],
       [4.19389, 0.     ]])

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

X_train = torch.from_numpy(X_train.to_numpy()).float()
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([2324, 2]) torch.Size([2324])
torch.Size([582, 2]) torch.Size([582])


In [ ]:
#data.to_csv (r'/Users/jithinjomy/Downloads/PHY661k/data1.csv', index = False, header=True)

In [14]:


class Model(nn.Module):
    def __init__(self, n_features):
        super(Model, self).__init__()
        self.lin1 = nn.Linear(n_features, 25)
        self.lin2 = nn.Linear(25, 50)
        self.lin3 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = torch.tanh(self.lin1(x))
        x = torch.tanh(self.lin2(x))
        return torch.sigmoid(self.lin3(x))

net =  Model(X_train.shape[1])
print(net)


Model(
  (lin1): Linear(in_features=2, out_features=25, bias=True)
  (lin2): Linear(in_features=25, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=1, bias=True)
)


In [15]:
import torch.optim as optim
import time

loss_criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
net = net.to(device)
loss_criterion = loss_criterion.to(device)


In [19]:
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)

for epoch in range(1000):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = loss_criterion(y_pred, y_train)
    
    if epoch % 10 == 0:
      train_acc = calculate_accuracy(y_train, y_pred)
      y_test_pred = net(X_test)
      y_test_pred = torch.squeeze(y_test_pred)
      test_loss = loss_criterion(y_test_pred, y_test)
      test_acc = calculate_accuracy(y_test, y_test_pred)
      print(
f'''epoch {epoch}
Train set - loss: {train_loss}, accuracy: {train_acc}
Test  set - loss: {test_loss}, accuracy: {test_acc}
''')

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    

  

epoch 0
Train set - loss: 0.07017350196838379, accuracy: 0.251290887594223
Test  set - loss: 0.06557746976613998, accuracy: 0.25601375102996826

epoch 10
Train set - loss: 0.07017022371292114, accuracy: 0.251290887594223
Test  set - loss: 0.06557124108076096, accuracy: 0.25601375102996826

epoch 20
Train set - loss: 0.07016696035861969, accuracy: 0.251290887594223
Test  set - loss: 0.06557241827249527, accuracy: 0.25601375102996826

epoch 30
Train set - loss: 0.07016374915838242, accuracy: 0.251290887594223
Test  set - loss: 0.06557122617959976, accuracy: 0.25601375102996826

epoch 40
Train set - loss: 0.07016053795814514, accuracy: 0.251290887594223
Test  set - loss: 0.06557074934244156, accuracy: 0.25601375102996826

epoch 50
Train set - loss: 0.07015744596719742, accuracy: 0.251290887594223
Test  set - loss: 0.0655788779258728, accuracy: 0.25601375102996826

epoch 60
Train set - loss: 0.07015934586524963, accuracy: 0.251290887594223
Test  set - loss: 0.06566306948661804, accuracy: 0

epoch 610
Train set - loss: 0.07000021636486053, accuracy: 0.251290887594223
Test  set - loss: 0.06545888632535934, accuracy: 0.25601375102996826

epoch 620
Train set - loss: 0.06999776512384415, accuracy: 0.251290887594223
Test  set - loss: 0.06545701622962952, accuracy: 0.25601375102996826

epoch 630
Train set - loss: 0.06999535858631134, accuracy: 0.251290887594223
Test  set - loss: 0.06544607877731323, accuracy: 0.25601375102996826

epoch 640
Train set - loss: 0.06999294459819794, accuracy: 0.251290887594223
Test  set - loss: 0.06545167416334152, accuracy: 0.25601375102996826

epoch 650
Train set - loss: 0.06999054551124573, accuracy: 0.251290887594223
Test  set - loss: 0.06544561684131622, accuracy: 0.25601375102996826

epoch 660
Train set - loss: 0.06998814642429352, accuracy: 0.251290887594223
Test  set - loss: 0.0654461532831192, accuracy: 0.25601375102996826

epoch 670
Train set - loss: 0.0699857622385025, accuracy: 0.251290887594223
Test  set - loss: 0.06544283777475357, accu

In [21]:
Voltage = 4.2
Current = 2.6
with torch.no_grad():
    t = torch.as_tensor([Voltage, Current])
    output = net(t)
    print(output)  

tensor([0.7216])


In [ ]:
MODEL_PATH = 'model.pth'
torch.save(net, MODEL_PATH)